In [1]:
import numpy as np
import pandas as pd
import sys
import pymongo
import functools
import json
import os
from pymongo import MongoClient

In [2]:
client = MongoClient()
db = client.credit_db

In [3]:
german_csv = pd.read_csv('German_Data/german.data',delimiter=' ')

In [4]:
german_csv.tail()

,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1.1
994,A14,12,A32,A42,1736,A61,A74,3,A92,A101,...,A121,31,A143,A152,1,A172,1,A191,A201,1
995,A11,30,A32,A41,3857,A61,A73,4,A91,A101,...,A122,40,A143,A152,1,A174,1,A192,A201,1
996,A14,12,A32,A43,804,A61,A75,4,A93,A101,...,A123,38,A143,A152,1,A173,1,A191,A201,1
997,A11,45,A32,A43,1845,A61,A73,4,A93,A101,...,A124,23,A143,A153,1,A173,1,A192,A201,2
998,A12,45,A34,A41,4576,A62,A71,3,A93,A101,...,A123,27,A143,A152,1,A173,1,A191,A201,1


In [5]:
casting_dict = {
'A11' : '... < 0 DM',
'A12' : '0 <= ... < 200 DM' ,
'A13' : '... >= 200 DM / salary assignments for at least 1 year', 
'A14' : 'no checking account' ,
'A30' : 'no credits taken/ all credits paid back duly' ,
'A31' : 'all credits at this bank paid back duly' ,
'A32' : 'existing credits paid back duly till now' ,
'A33' : 'delay in paying off in the past' ,
'A34' : 'critical account/ other credits existing (not at this bank)' ,
'A40' : 'car (new)' ,
'A41' : 'car (used)' ,
'A42' : 'furniture/equipment',
'A43' : 'radio/television' ,
'A44' : 'domestic appliances' ,
'A45' : 'repairs' ,
'A46' : 'education' ,
'A47' : '(vacation - does not exist?)' ,
'A48' : 'retraining' ,
'A49' : 'business' ,
'A410' : 'others' ,
'A61' : '... < 100 DM' ,
'A62' : '100 <= ... < 500 DM' ,
'A63' : '500 <= ... < 1000 DM ',
'A64' : '.. >= 1000 DM' ,
'A65' : 'unknown/ no savings account' ,
'A71' : 'unemployed' ,
'A72' : '... < 1 year' ,
'A73' : '1 <= ... < 4 years ',
'A74' : '4 <= ... < 7 years' ,
'A75' : '.. >= 7 years' ,
'A91' : 'male : divorced/separated' ,
'A92' : 'female : divorced/separated/married' ,
'A93' : 'male : single' ,
'A94' : 'male : married/widowed ',
'A95' : 'female : single' ,
'A101' : 'none' ,
'A102' : 'co-applicant' ,
'A103' : 'guarantor' ,
'A121' : 'real estate' ,
'A122' : 'if not A121 : building society savings agreement/ life insurance' ,
'A123' : 'if not A121/A122 : car or other, not in attribute 6',
'A124' : 'unknown / no property',
'A141' : 'bank' ,
'A142' : 'stores' ,
'A143' : 'none',
'A151' : 'rent' ,
'A152' : 'own' ,
'A153' : 'for free' ,
'A171' : 'unemployed/ unskilled - non-resident' ,
'A172' : 'unskilled - resident' ,
'A173' : 'skilled employee / official' ,
'A174' : 'management/ self-employed/ highly qualified employee/ officer' ,
'A191' : 'none' ,
'A192' : 'yes, registered under the customers name',
'A201' : 'yes' ,
'A202' : 'no' 
}

In [6]:
key_df = pd.DataFrame.from_dict(casting_dict,orient='index')
key_df  = key_df.reset_index()
key_df.columns = [['key','desc']]

In [7]:
key_df['key'] = key_df['key'].astype(str)

In [8]:
german_csv['A11'] = german_csv['A11'].astype(str)

In [9]:
german_df  = german_csv.replace(casting_dict)


In [10]:
german_df.columns = ['Status of existing checking account',
                     'Duration in month',
                     'Credit history',
                     'Purpose',
                     'Credit amount',
                     'Savings account/bonds',
                     'Present employment since',
                     'Installment rate in percentage of disposable income',
                     'Personal status and sex',
                     'Other debtors / guarantors',
                     'Present residence since',
                     'Property',
                     'Age in years',
                     'Other installment plans',
                     'Housing',
                     'Number of existing credits at this bank',
                     'Job',
                     'Number of people being liable to provide maintenance for',
                     'Telephone',
                     'foreign worker',
                     'na']

In [11]:
german_df['country'] = '2'

In [12]:
german_df.drop(columns='na',inplace=True)

In [13]:
german_df.tail(2)

,Status of existing checking account,Duration in month,Credit history,Purpose,Credit amount,Savings account/bonds,Present employment since,Installment rate in percentage of disposable income,Personal status and sex,Other debtors / guarantors,...,Property,Age in years,Other installment plans,Housing,Number of existing credits at this bank,Job,Number of people being liable to provide maintenance for,Telephone,foreign worker,country
997,... < 0 DM,45,existing credits paid back duly till now,radio/television,1845,... < 100 DM,1 <= ... < 4 years,4,male : single,none,...,unknown / no property,23,none,for free,1,skilled employee / official,1,"yes, registered under the customers name",yes,2
998,0 <= ... < 200 DM,45,critical account/ other credits existing (not ...,car (used),4576,100 <= ... < 500 DM,unemployed,3,male : single,none,...,"if not A121/A122 : car or other, not in attrib...",27,none,own,1,skilled employee / official,1,none,yes,2


In [14]:
# Taiwan Data
default_credit_xls = pd.read_excel('taiwan_data/default of credit card clients.xls')

In [29]:
default_credit_df = default_credit_xls.drop(columns=['Unnamed: 0'])
default_credit_df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


In [16]:
default_credit_df.columns = default_credit_df.iloc[0,:]
# We set this to a new varitable because if you run it again taking 1loc off every time you lose one row at a time.
cleaning_df = default_credit_df[1:]
dc_taiwan_df = cleaning_df.copy()
dc_taiwan_df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [17]:
dc_taiwan_df.loc[dc_taiwan_df['SEX'] == 1, 'SEX'] = 'male'
dc_taiwan_df.loc[dc_taiwan_df['SEX'] == 2, 'SEX'] = 'female'
dc_taiwan_df.loc[dc_taiwan_df['EDUCATION'] == 1, 'EDUCATION'] = 'graduate school'
dc_taiwan_df.loc[dc_taiwan_df['EDUCATION'] == 2, 'EDUCATION'] = 'university'
dc_taiwan_df.loc[dc_taiwan_df['EDUCATION'] == 3, 'EDUCATION'] = 'high school'
dc_taiwan_df.loc[dc_taiwan_df['EDUCATION'] == 4, 'EDUCATION'] = 'others'
dc_taiwan_df.loc[dc_taiwan_df['MARRIAGE'] == 1, 'MARRIAGE'] = 'married'
dc_taiwan_df.loc[dc_taiwan_df['MARRIAGE'] == 2, 'MARRIAGE'] = 'single'
dc_taiwan_df.loc[dc_taiwan_df['MARRIAGE'] == 3, 'MARRIAGE'] = 'others'

In [18]:
dc_taiwan_df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,female,university,married,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,female,university,single,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,female,university,single,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,female,university,married,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,male,university,married,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [19]:
credit_card_df = pd.read_csv('USA_default_data/Credit Card Loans.csv',index_col='DATE')
farmland_df = pd.read_csv('USA_default_data/Farmland Loans.csv',index_col='DATE')
other_df = pd.read_csv('USA_default_data/Other Consumer Loans.csv',index_col='DATE')
single_family_df = pd.read_csv('USA_default_data/Single-Family Residential Mortgages.csv',index_col='DATE')

In [20]:
# Wow this actually worked with out having to strftime to hell
credit_card_df.index = pd.to_datetime(credit_card_df.index)
farmland_df.index = pd.to_datetime(farmland_df.index)
other_df.index = pd.to_datetime(other_df.index)
single_family_df.index = pd.to_datetime(single_family_df.index)


In [21]:
dfs = [credit_card_df,farmland_df,other_df,single_family_df]

In [22]:
us_default_rates = functools.reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True), dfs)

In [23]:
column_names = ['Credit Card','Farmland','Other','Single Family']
us_default_rates.columns = column_names

In [24]:
us_default_rates.head()

,Credit Card,Farmland,Other,Single Family
DATE,,,,
1991-01-01,5.31,4.86,3.52,3.18
1991-04-01,5.35,4.18,3.41,3.07
1991-07-01,5.43,4.05,3.54,3.17
1991-10-01,5.32,4.16,3.66,3.30
1992-01-01,5.32,4.88,3.24,3.23


In [25]:
USA_recods = json.loads(us_default_rates.T.to_json()).values()
taiwan_recods = json.loads(dc_taiwan_df.T.to_json()).values()
german_recods = json.loads(german_df.T.to_json()).values()

db.USA.insert_many(USA_recods)
db.Taiwan.insert_many(taiwan_recods)
db.Germany.insert_many(german_recods)


dtype('<M8[ns]')